In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
!time conda install -q -y -c openbabel openbabel

In [6]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [7]:
!pip install py3Dmol # 3D Molecular Visualizer

In [ ]:
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import DataStructs
from rdkit.Chem import RDConfig
from rdkit.Chem import rdBase
import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(rdBase.rdkitVersion)

In [ ]:
!wget https://sourceforge.net/projects/smina/files/smina.static/download -O smina.static

In [10]:
!chmod u+x smina.static

In [ ]:
!./smina.static

In [ ]:
! apt-get install pymol 

In [ ]:
!wget -O 1OYT.pdb https://files.rcsb.org/download/1oyt.pdb
com_file = open('fetch_and_clean.pml','w')
com_file.write('''
load 1OYT.pdb
remove resn HOH
h_add elem O or elem N
select 1OYT-FSN, resn FSN #Create a selection called 1OYT-FSN from the ligand
select 1OYT-receptor, 1OYT and not 1OYT-FSN #Select all that is not the ligand
save 1OYT-FSN.pdb, 1OYT-FSN
save 1OYT-receptor.pdb, 1OYT-receptor    
''')
com_file.close()

In [ ]:
!pymol -c fetch_and_clean.pml

In [ ]:
!obabel 1OYT-receptor.pdb -xr -O 1OYT-receptor.pdbqt
!obabel 1OYT-FSN.pdb -O 1OYT-FSN.pdbqt

In [ ]:
!./smina.static -r 1OYT-receptor.pdbqt -l 1OYT-FSN.pdbqt --autobox_ligand 1OYT-FSN.pdbqt --autobox_add 8 --exhaustiveness 16 -o 1OYT-redock.pdbqt

In [17]:
import py3Dmol

def drawit2(m,confId=-1):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p

def DrawComplex(protein,ligand):
    complex_pl = Chem.MolToPDBBlock(Chem.CombineMols(protein,ligand))
    #complex_mol=Chem.CombineMols(receptor,mols[-1])
    view = py3Dmol.view(width=600,height=600)
    view.addModel(complex_pl,'pdb')
    #view.addModel(Chem.MolToMolBlock(mols[0]),'sdf')
    chA = {'chain':['H','L','I']}
    chB = {'resn':'UNL'}
    view.setStyle(chA,{'cartoon': {'color':'spectrum'}})
    #view.setStyle(chA,{'lines': {}})
    view.addSurface(py3Dmol.VDW,{'opacity':0.8}, chB)
    view.setStyle(chB,{'stick':{}})
    view.zoomTo()
    return view   


In [18]:
from rdkit import Chem

In [19]:
!obabel 1OYT-redock.pdbqt -O 1OYT-redocked.sdf

9 molecules converted


In [20]:
mols = [m for m in Chem.SDMolSupplier('1OYT-redocked.sdf') if m is not None]

In [ ]:
drawit2(mols[8])

In [ ]:
receptor = Chem.MolFromPDBFile('1OYT-receptor.pdb')
DrawComplex(receptor,mols[8])